In [24]:
from bs4 import BeautifulSoup
from struct import *
import urllib.request
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import os
import sys
import time
import threading
import re
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from subprocess import STDOUT, PIPE, Popen, check_output, check_call, call
import socket
import netifaces as ni
import string

# get local ip address
ni.ifaddresses('en0')
myip = ni.ifaddresses('en0')[2][0]['addr']

# get keyword dictionary 
K=list()
with open('keywords.txt','r') as f:
    for line in f:
        if len(line)>3:
            K.append(line[:-1])

def terminal(cmd):
    os.system(cmd)

# def tshark(duration,filename):
#     cmd="""tshark -Y "((ip.dst_host == """+myip+""" && ip.src_host ==213.180.204.62  )|| (ip.dst_host == 213.180.204.62&& ip.src_host == """+myip+"""  ) )&&tcp.port == 443 && ssl.record.version == 0x0303" -T fields -e frame.time -e ip.src -e ip.dst -e frame.len -e _ws.col.Info -a duration:"""+str(duration)+""" > """ + filename
#     t=threading.Thread(target=terminal,args=([cmd]))
#     t.start()
#     return t

def mitmproxy(duration):
    cmd="""mitmdump -s 'dumpfile.py yandex\.com' """
    t=threading.Thread(target=terminal,args=([cmd]))
    t.start()
    #time.sleep(duration)
    time.sleep(duration)
    return t      
             
def enterAWord(wrd,inputElement):
   # driver.get("https://yandex.com")
   # inputElement = driver.find_element_by_id("text")
   # time.sleep(1)
    for c in wrd:
        inputElement.send_keys(c)
        time.sleep(1)


def enterAllAlphabet(inputElement):
   
   # ts2=tshark(9,'dump1.csv')
    time.sleep(1)
    alphabet=list(string.ascii_lowercase)
    for c in alphabet:
        inputElement.send_keys(c)
        bs=Keys.BACKSPACE
        time.sleep(0.5)
        inputElement.send_keys(bs)
        time.sleep(0.5)
    #ts2.join()
    time.sleep(30)
    T2=readProxyDump('sadu123.txt')
    return T2

def clearSearch(inputElement):
    inputElement.clear()
    

def readProxyDump(file):
    f=open(file,'r')
    dump=''.join([line for line in f])
    r=re.findall("text/javascript;\scharset=UTF-8,\s+(\d+)b",dump)
    return list(r)


def deleteContent(fName):
    with open(fName, "w"):
        pass
    
def findMatches(aList,func):
    return [i for (i, val) in enumerate(aList) if func(val)]

def openYandex():
    driver=webdriver.Chrome()
    driver.get("https://yandex.com")
    time.sleep(2)
    inputElement = driver.find_element_by_id("text")
    time.sleep(1)
    return inputElement



In [5]:
myip

'192.168.0.105'

In [25]:
deleteContent('sadu123.txt')


In [27]:
duration=20  # assume 10 sec enough to catch userinput 
tp=mitmproxy(duration) # file name is specified inside dumpfile.py "sadu123.txt" for the time being
time.sleep(2)

In [30]:
T=readProxyDump('sadu123.txt')
print(T)

['62', '751', '930', '751', '494', '751', '62', '751', '249', '751', '230', '751', '62', '751', '62', '751', '62', '751', '62', '62', '212', '62', '62', '751', '357', '751', '62', '751', '208', '751', '62', '751', '62', '751']


In [29]:
tree=[]
topofTree={'char':'null', 'parent':-1, 'degree':0, 'flag':1} #flag shows if node is active or removed
# topofTree['parent']
tree.append(topofTree)
#tree[0]['parent']

userInput=T
userInputSize=len(userInput)
i = 0
alphabet=list(string.ascii_lowercase)

#OPEN CHROME w8 for 3 sec
currentDegree=0


while(i<userInputSize):
    currentPackSize=userInput[i]
    inserted=0 # check if new node inserted or not to increment degree
    newNodesOfTree=[] #new Nodes to be inserted into the tree at the end of 26 letter type process
    #if first letter, directly type all letters, insert matches
    if( i==0 ):
        i = i + 1
        inputElement = openYandex() 
        deleteContent('sadu123.txt') #delete contents
        time.sleep(2)
        letterPackSizes = enterAllAlphabet(inputElement) #type 26 letter
        print(letterPackSizes) 
        #insert matches to the tree
        indices = findMatches(letterPackSizes,lambda x:x==userInput[0])
        for x in range(0,len(indices)):
           # print(alphabet[indices[x]])
            newNode = {'char':alphabet[indices[x]],'parent':0,'degree':1,'flag':1}
            tree.append(newNode)
            #print(alphabet[indices[x]])
        currentDegree += 1
        continue
    
    
   # print(letterPackSizes)
   # print(tree)
    # number of element that have i degree
    rootNum=len([t for t in range(len(tree)) if tree[t]['degree']==currentDegree])

    for j in range (rootNum,0,-1):
        newNode=tree[len(tree)-j]
        #if(newNode['flag']==0) continue # if removed node no need to go on
        toBeTypedBefore=[]
        currentNode=newNode
        print("CurrentNode:")
        print(currentNode)
        while( currentNode['parent'] != -1 ):
            toBeTypedBefore.append(currentNode['char']) # bottom one inserted
            currentNode=tree[currentNode['parent']]
        currentNode=newNode
        toBeTypedBefore=toBeTypedBefore[::-1]
        print("toBeTypedBefore:" + ''.join(toBeTypedBefore))  #type reverse of toBeTypedBefore
        
        clearSearch(inputElement)  #clear search engine
        enterAWord(toBeTypedBefore,inputElement)        #type toBeTyped
        deleteContent('sadu123.txt') #delete contents
        time.sleep(2)
        
        print("PRINTING ALL ALPHABET 1")
        letterPackSizes=enterAllAlphabet(inputElement) #type 26 letter after that 
        print("letterPackSizes" + repr(letterPackSizes) + "\n")
        letterPackSizesTemp=[]
        
        for k in range (0,len(letterPackSizes),2): #remove repetitive package sizes
            letterPackSizesTemp.append(letterPackSizes[k])
        letterPackSizes=letterPackSizesTemp
        print('letterPackSizes' + repr(letterPackSizes) +'\n')

        indices=findMatches(letterPackSizes,lambda x:x==userInput[i]) #find matches
        print('indices' + repr(indices))

        tempNode=[]
        
        if(len(indices)==0): #if there is no match kill that path
          # while( currentNode['parent'] != -1 ): #remove recursively
            tempNode=tree[currentNode['parent']]
            print(repr(currentNode) + '-> flag set to 0:')
            index=tree.index(currentNode) #find location of currentnode
            tree[index]['flag']=0
            currentNode=tempNode
        else :#else insert matches to that node
        
            print('indices' + repr(len(indices)))
            if(len(indices) == 26):
                print('SORRY ADMIN... NO MORE REDUCTION in this path ')
                continue
            else:  
                inserted=1
                for x in range(0,len(indices)):
                    newNode={'char':alphabet[indices[x]],'parent':tree.index(currentNode),'degree':(currentDegree+1),'flag':1}
                    newNodesOfTree.append(newNode)
                    print("newNodesOfTree:")
                    print(newNodesOfTree)
    
            
   
    if(inserted):
        currentDegree=currentDegree+1
    
    print('Number of Nodes:' + repr(len(newNodesOfTree)))
    tree=tree+newNodesOfTree
    
    
    
    i=i+1 
# Print In Reverse Order
#s='asd'
#s[::-1]

['759', '770', '776', '768', '753', '763', '823', '770', '758', '782', '746', '765', '751', '798', '790']
CurrentNode:
{'char': 'f', 'parent': 0, 'degree': 1, 'flag': 1}
toBeTypedBefore:f
PRINTING ALL ALPHABET 1
letterPackSizes['860', '763', '749', '763', '762', '763', '711', '763', '790', '715', '705', '747', '783', '765', '760', '763', '781', '763', '747', '763', '763', '763', '769', '763', '766', '763', '720', '763', '751', '763']

letterPackSizes['860', '749', '762', '711', '790', '705', '783', '760', '781', '747', '763', '769', '766', '720', '751']

indices[0]
indices1
newNodesOfTree:
[{'char': 'a', 'parent': 1, 'degree': 2, 'flag': 1}]
Number of Nodes:1
CurrentNode:
{'char': 'a', 'parent': 1, 'degree': 2, 'flag': 1}
toBeTypedBefore:fa
PRINTING ALL ALPHABET 1
letterPackSizes['746', '796', '860', '748', '751', '815', '860', '782', '860', '843', '860', '813', '860', '808', '860', '822', '860', '722', '860', '789', '860', '466', '860', '795', '799', '889', '798', '860', '779', '860',

KeyboardInterrupt: 